In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.request

import time
import pandas as pd
import scipy as sc
import numpy as np

import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6) 
from bs4 import UnicodeDammit


In [2]:
#Pulls the grade lists from the csv files, combines the dataframes for the same semester
#and removes the suppressed data
def getgrades(csv1, csv2):
    grades1=pd.read_csv(csv1, encoding="utf8")
    grades2=pd.read_csv(csv2, encoding="utf8")
    grds=[grades1, grades2]
    grades=pd.concat(grds,ignore_index=True)
    i=0
    grades=grades.reset_index(drop=True)
    hc=grades['sumHeadcount']
    while i < len(hc):
        if hc[i]== 'ds':
            grades=grades.drop(i, axis=0)
        i+=1
    grades=grades.reset_index(drop=True)
    return(grades)

In [3]:
#Combines the grades of each class into one row
def combinegrades(grades):
    i=1
    n=2
    gradesclean=pd.DataFrame([],columns=['Num', 'Section', 'Subject', 'A', 'B', 'C', 'D', 'E', 'W', 'Other'])
    gradesclean=gradesclean.append({'Num':1050, 'Section':1, 'Subject':'ASTR - Astronomy', 'A':0, 'B':0, 'C':0, 'D':0,
                                    'E':0, 'W':0, 'Other':2}, ignore_index=True)
    while i<len(grades['sumHeadcount']):
        if grades['Subject'].iloc[i]==gradesclean['Subject'].iloc[-1]:
            if grades['Catalog Num'].iloc[i]==gradesclean['Num'].iloc[-1]:
                fail=0
                for m in range(1,n):
                    if grades['Section'].iloc[i]==gradesclean['Section'].iloc[-m]:
                        gradesclean[grades['Grade Group'].iloc[i]].iat[-m]=grades['sumHeadcount'].iloc[i]
                        break
                    fail+=1
                if fail==n-1:
                    n+=1
                    gradesclean=gradesclean.append({'Num':grades['Catalog Num'].iloc[i],
                                                    'Section':grades['Section'].iloc[i],
                                                    'Subject':grades['Subject'].iloc[i], 'A':0, 'B':0,
                                                    'C':0,'D':0, 'E':0, 'W':0, 'Other':0}, ignore_index=True)
                    gradesclean[grades['Grade Group'].iloc[i]].iat[-1]=grades['sumHeadcount'].iloc[i]
            else:
                n=2
                gradesclean=gradesclean.append({'Num':grades['Catalog Num'].iloc[i], 'Section':grades['Section'].iloc[i],
                                                'Subject':grades['Subject'].iloc[i], 'A':0, 'B':0,
                                                'C':0,'D':0, 'E':0, 'W':0, 'Other':0}, ignore_index=True)
                gradesclean[grades['Grade Group'].iloc[i]].iat[-1]=grades['sumHeadcount'].iloc[i]
        else:
            n=2
            gradesclean=gradesclean.append({'Num':grades['Catalog Num'].iloc[i], 'Section':grades['Section'].iloc[i],
                                            'Subject':grades['Subject'].iloc[i], 'A':0, 'B':0,
                                            'C':0,'D':0, 'E':0, 'W':0, 'Other':0}, ignore_index=True)
            gradesclean[grades['Grade Group'].iloc[i]].iat[-1]=grades['sumHeadcount'].iloc[i]
        i+=1
    return(gradesclean)

In [5]:
#The only code needed to run each time more data is added, 
#to add more just add to the sgrades/egrades lists and lengthen cleaned
sgrades=['fall17sci.csv','spring17sci.csv']
egrades=['fall17eng.csv','spring17eng.csv']
i=0
cleaned=[0,0]
while i<len(sgrades):
    gradess=getgrades(sgrades[i],egrades[i])
    gradescleans=combinegrades(gradess)
    cleaned[i]=gradescleans
    i+=1


In [ ]:
cleaned[1]